In [ ]:
from ricehealthai.core.settings import load_config
from ricehealthai.infrastructure.logger import setup_logger

logger = setup_logger(__name__)
data_cfg = load_config("configs/data_config.yaml")
logger.info(f"ressource path: {data_cfg['data']['raw_dir']}")



if __name__ == "__main__":
    main()

2025-10-16 14:02:44 | INFO     | __main__:main - Image size: [224, 224]
